TODO:
- Figure out how to properly implement rebalancing
- Add user input reading to determine parameters

## Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
import seaborn as sns
import math
import pandas_market_calendars as mcal

ModuleNotFoundError: No module named 'pandas_market_calendars'

### Parameters

- All parameters below will be selected via user inputs, for now I'm setting them as constants

In [48]:
# Tickers of ETFS and their portfolio weighings
products = {
          "SPY":0.4,    #SP500
          "QQQ":0.2,   #NASDAQ100
          "IWM":0.0,    #Russel2000
           #Bonds
          "BIL":0.0,     #0-3 month 
          "SHY":0.4,    #1-3 yr 
          "IEF":0.0,      #7-10yr
          "TLT":0.0       #20+yr
}

# Start and end dates for the test 
start_date = '2010-01-01'
end_date = '2023-01-01'

# Deposit frequency and amount
# ["daily","weekly","monthly","quarterly","yearly"] -> [1,7,30,120,365]
frequency = 'BMS' # Deposit frequency
# ['B':business day,'BM':business month end, 'BMS':business month start
# 'BQ': business quarter end, 'BQS': Business quarter start
# 'BA,BY': busines year end, 'BAS,BYS': buiness year start
deposit = 100 # Deposit amount
  
# Rebalancing?
rebalancing_frequency = 'BYS' # How many times a year

Fill in this functiion after writing the code part by part

In [49]:
def generate_deposits_dataframe(products, start_date, end_date, frequency, deposit, rebalancing_freq):
    # Generate time series with given frequency between start and end date
    dates = pd.date_range(start_date, end_date, freq=frequency)
    df = pd.DataFrame({'date': dates})

Generate dates

In [50]:
# Generate initial deposit dates and put them in a dataframe
deposit_dates = pd.date_range(start_date, end_date, freq=frequency)
df = pd.DataFrame({'Dates': deposit_dates})
df["Deposit"] = True

### Make sure none of them are market holidays

# Create a calendar
nyse = mcal.get_calendar('NYSE')

# Function to find the next trading day
def next_trading_day(date):
    while True:
        date = date + pd.Timedelta(days=1)
        if nyse.valid_days(start_date=date, end_date=date).size > 0:
            return date

# Replace dates that are US market holidays or weekend days with the nearest trading day
df['Dates'] = df['Dates'].apply(lambda x: next_trading_day(x) if nyse.valid_days(start_date=x, end_date=x).size == 0 or x.weekday() >= 5 else x)

# Extra check on modified dates: Check if each date in the "Dates" column is a US market holiday or a weekend day
df['Holiday_or_Weekend'] = df['Dates'].apply(lambda x: 'Holiday' if nyse.valid_days(start_date=x, end_date=x).size == 0 else 'Weekend' if x.weekday() >= 5 else 'Trading Day')


# Add rebalancing dates based on specified rebalancing frequency

# Create a date range with the specified rebalancing frequency
rebalance_dates = pd.date_range(start_date, end_date, freq=rebalancing_frequency).tolist() 
df_rebalancing = pd.DataFrame({'Dates': rebalance_dates}) 

# Create a new column "Rebalancing" in df and initialize it to False
df['Rebalancing'] = False

# Loop through each date in df_rebalancing and mark the corresponding date in df as rebalancing - Very slow
for date in df_rebalancing['Dates']:
    # Find the index of the matching date in df, or the index of the closest date if no match exists
    idx = df['Dates'].sub(date).abs().idxmin()
    # Set the corresponding row in df to be a rebalancing date
    df.loc[idx, 'Rebalancing'] = True


Add ticker info

In [51]:
# Add each desired ticker with its weight to the dataframe

# Create a list of lists with the ticker and its weight
tickers = []
for key, value in products.items():
    if value != 0:
        tickers.append([key, value])


# The below columns represent how many $ have been spent on each ticker at each date
for ticker in tickers:
    col_name = f"{ticker[0]}_spent"
    df[col_name] = ticker[1] * deposit


# The below columns represent how much each ticker cost at each date
for ticker in tickers:
    ticker_price_column = f"{ticker[0]}_price"
    df[ticker_price_column ] = 0
    
    # Get historical prices for our given dates from yahoo finance
    ticker_obj = yf.Ticker(ticker[0])
    history = ticker_obj.history(start=start_date, end=end_date) 
    
    date_list = df['Dates'].dt.strftime('%Y-%m-%d') # get our list of investment dates
    prices = history.loc[date_list]['Close'] # If this line gives an error some of our dates aren't market days
    # TODO: Add catch error line

    df[ticker_price_column] = prices.to_list()





In [52]:
# Add column to indicate number of shares bought at each date, cumulative number of shares bought at each date, 
for ticker in tickers:
    
    new_shares_bought = f"{ticker[0]}_bought"
    df[new_shares_bought] = df[f"{ticker[0]}_spent"] / df[f"{ticker[0]}_price"]

    shares_bought_cumulative = f"{ticker[0]}_bought_cum"
    df[shares_bought_cumulative] = df[new_shares_bought].cumsum()

    shares_value_cumulative = f"{ticker[0]}_value_cum"
    df[shares_value_cumulative] = df[shares_bought_cumulative] * df[f"{ticker[0]}_price"]


In [53]:
# Implement rebalancing



,Dates,Deposit,Holiday_or_Weekend,Rebalancing,SPY_spent,QQQ_spent,SHY_spent,SPY_price,QQQ_price,SHY_price,SPY_bought,SPY_bought_cum,SPY_value_cum,QQQ_bought,QQQ_bought_cum,QQQ_value_cum,SHY_bought,SHY_bought_cum,SHY_value_cum
0,2010-01-04,True,Trading Day,True,40.0,20.0,40.0,88.117882,41.005466,73.689186,0.453937,0.453937,40.000000,0.487740,0.487740,20.000000,0.542820,0.542820,40.000000
1,2010-02-01,True,Trading Day,False,40.0,20.0,40.0,84.797821,38.214043,74.145058,0.471710,0.925648,78.492900,0.523368,1.011108,38.638511,0.539483,1.082304,80.247456
2,2010-03-01,True,Trading Day,False,40.0,20.0,40.0,86.998253,40.113274,74.282700,0.459779,1.385427,120.529725,0.498588,1.509696,60.558838,0.538483,1.620787,120.396427
3,2010-04-01,True,Trading Day,False,40.0,20.0,40.0,91.970657,42.587921,74.069374,0.434921,1.820348,167.418628,0.469617,1.979312,84.294803,0.540034,2.160821,160.050672
4,2010-05-03,True,Trading Day,False,40.0,20.0,40.0,93.961533,44.153126,74.245438,0.425706,2.246054,211.042714,0.452969,2.432282,107.392832,0.538754,2.699575,200.431113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,2022-08-01,True,Trading Day,False,40.0,20.0,40.0,405.671326,313.467529,81.757919,0.098602,35.652040,14463.010219,0.063802,32.471771,10178.845878,0.489249,77.368067,6325.452152
152,2022-09-01,True,Trading Day,False,40.0,20.0,40.0,391.499451,297.688263,81.038429,0.102171,35.754211,13997.753968,0.067184,32.538956,9686.465152,0.493593,77.861660,6309.786598
153,2022-10-03,True,Trading Day,False,40.0,20.0,40.0,363.546722,272.454926,80.306519,0.110027,35.864238,13038.326223,0.073407,32.612362,8885.398708,0.498092,78.359751,6292.798816
154,2022-11-01,True,Trading Day,False,40.0,20.0,40.0,381.307068,274.028687,79.951691,0.104902,35.969140,13715.287482,0.072985,32.685347,8956.722771,0.500302,78.860053,6304.994591


In [ ]:
# Add column to indicate cumulative number of shares bought at each date, along with present value of each ticker
for ticker in tickers:
    new_shares_bought = f"{ticker[0]}_shares_bought"
    
    df[new_ticker_value] = df[shares_bought_cumulative] * df[f"{ticker[0]}_price"]

In [16]:
# Get prices for our given dates from yahoo finance
ticker_obj = yf.Ticker('SHY')
history = ticker_obj.history(start=start_date, end=end_date) # get historical prices

date_list = df['Dates'].dt.strftime('%Y-%m-%d') # get our list of investment dates

prices = history.loc[date_list]['Close'] # If this line gives an error some of our dates aren't market days

In [36]:
prices.to_list()

[73.68910217285156,
 74.14508819580078,
 74.2826919555664,
 74.06937408447266,
 74.24551391601562,
 74.65597534179688,
 74.94377899169922,
 75.09629821777344,
 75.2480239868164,
 75.39543151855469,
 75.58052825927734,
 75.27982330322266,
 75.25125885009766,
 75.28524780273438,
 75.32209777832031,
 75.23860168457031,
 75.56745910644531,
 75.90556335449219,
 75.82543182373047,
 76.0423812866211,
 76.3523941040039,
 76.27033233642578,
 76.2865982055664,
 76.31822967529297,
 76.30823516845703,
 76.39046478271484,
 76.29553985595703,
 76.26842498779297,
 76.40231323242188,
 76.45481872558594,
 76.39684295654297,
 76.52912902832031,
 76.54177856445312,
 76.56172943115234,
 76.48916625976562,
 76.53448486328125,
 76.52633666992188,
 76.54536437988281,
 76.61618041992188,
 76.63248443603516,
 76.69326782226562,
 76.59980773925781,
 76.53352355957031,
 76.59529876708984,
 76.52078247070312,
 76.70064544677734,
 76.74515533447266,
 76.79872131347656,
 76.74515533447266,
 76.88420867919922,
 76.9

### Function to build dataframe with required parameters

In [4]:
def create_df(products,start_date,end_date,frequency,deposit,rebalancing,rebalancing_frequency):
    # Get data from yahoo finance
    tickers_use = sorted([key for key,value in products.items() if value != 0])
    #dates = pd.date_range(start_date, end_date,frequency='B')

    data = yf.download(tickers_use, start=start_date, end=end_date)
    df = pd.DataFrame(data['Adj Close'])
    df.reset_index(inplace=True)

    # Add Transact, Deposit, Rebalancing Flags
    df['Deposit'] = 0
    df['Transact'] = False
    df['Rebalance'] = False


    for i in range(0,len(df),frequency):
        df.at[i,'Transact'] = True
        df.at[i,'Deposit'] = deposit

    for i in range(rebalancing_frequency,len(df),rebalancing_frequency):
        df.at[i,'Rebalance'] = True


    #Calculate amount of each instrument bought on given date
    units_bought = [f'{ticker}_units_bought' for ticker in tickers_use ]
    units_cum = [f'{ticker}_units_cum' for ticker in tickers_use ]
    units_value = [f'{ticker}_value' for ticker in tickers_use ]

    for i in range(len(units_bought)): # For each product
        p = tickers_use[i]
        weight = products[p] 
        df[units_bought[i]] =  df.Deposit * weight / df[tickers_use[i]]

        df[units_cum[i]] = df[units_bought[i]].cumsum()

        weight = products[tickers_use[i]] 
        df[units_value[i]] =  df[tickers_use[i]] * df[units_cum[i]]
        
        #if df['Rebalance'][i] == True: ## Commence rebalancing 

    #Portfolio value and cumulative deposits
    df['Portfolio_Value'] = df[units_value].sum(axis=1)
    df['Deposits_cum'] = df.Deposit.cumsum()
    df['OverallPnL%'] = (df['Portfolio_Value'] - df['Deposits_cum'])/ df['Deposits_cum']
    #Figure out smart way to do this
    df['DoDPnL%'] = 0
    df['DoDLogReturn'] = 0
    for i in range(1,len(df)):
        df.at[i,'DoDPnL%'] = (df.at[i,'Portfolio_Value'] - df.at[i-1,'Portfolio_Value']-df.at[i,'Deposit'])/df.at[i-1,'Portfolio_Value']
        df['DoDLogReturn'] =  np.log((df.at[i,'Portfolio_Value'] -df.at[i,'Deposit']) / (df.at[i-1,'Portfolio_Value']-df.at[i,'Deposit']))
    
    return df
    

In [5]:
df = create_df(products,start_date,end_date,frequency,deposit,rebalancing,rebalancing_frequency)

[*********************100%***********************]  2 of 2 completed


In [6]:
df.tail()

,Date,SHY,SPY,Deposit,Transact,Rebalance,SHY_units_bought,SHY_units_cum,SHY_value,SPY_units_bought,SPY_units_cum,SPY_value,Portfolio_Value,Deposits_cum,OverallPnL%,DoDPnL%,DoDLogReturn
2511,2019-12-24,82.794525,307.631134,0,False,False,0.00000,183.802342,15217.827627,0.000000,142.526135,43845.476665,59063.304292,35900,0.645217,0.000054,0.00177
2512,2019-12-26,82.823921,309.268768,0,False,False,0.00000,183.802342,15223.230691,0.000000,142.526135,44078.882349,59302.113041,35900,0.651869,0.004043,0.00177
2513,2019-12-27,82.882713,309.192047,100,True,False,0.48261,184.284952,15274.036819,0.194054,142.720190,44127.947575,59401.984394,36000,0.650055,-0.000002,0.00177
2514,2019-12-30,82.912071,307.487457,0,False,False,0.00000,184.284952,15279.447040,0.000000,142.720190,43884.668189,59164.115229,36000,0.643448,-0.004004,0.00177
2515,2019-12-31,82.902283,308.234436,0,False,False,0.00000,184.284952,15277.643165,0.000000,142.720190,43991.277139,59268.920304,36000,0.646359,0.001771,0.00177


### Calculate Performance Metrics

Using Portfolio log returns

In [7]:
r = df.DoDLogReturn.to_numpy()
std = np.sqrt(np.sum(np.square(r)))
std /= np.sqrt(251)
std

0.0056034824620875565

#### Portfolio Volatility as weighted sum of individual securities
- This assumes the portfolio weights hold up

Plotting and stuff